# Debug de Componentes do Agente Multi-Camada (Full Stack)

Este notebook testa a integração completa.
Se o `debug_context_manager.ipynb` funcionou, este também deve funcionar se as importações estiverem atualizadas.

In [0]:
%load_ext autoreload
%autoreload 2
import os
import sys
from pyspark.sql import SparkSession

sys.path.append(os.getcwd())

In [0]:
dbutils.notebook.exit("Reinicie o notebook manualmente ou execute novamente todas as células.")

## 1. Inicializando Sessão & Data Engine
Verificamos se o Data Engine pega a sessão correta.

In [0]:
from app.data_engine import DataEngine

print("Inicializando DataEngine...")
data_engine = DataEngine()

# Pega a sessão (deve ser a mesma do debug anterior)
spark = data_engine.initialize_session()
print(f"Spark Session obtida: {spark}")

# Validação cruzada (Sanity Check)
active = SparkSession.getActiveSession()
if spark == active:
    print("[OK] DataEngine está usando a sessão ativa correta.")
else:
    print("[ALERTA] DataEngine criou ou pegou uma sessão diferente da ativa!")
    # Force usage of active if they differ to avoid issues
    spark = active

## 2. Teste do Context Manager (Schema Injection)
Deve imprimir o schema sem erros de Socket/Security.

In [0]:
from app.context_manager import ContextManager

context_mgr = ContextManager(spark)

role = "logistics"
print(f"Gerando contexto para role: {role}...")

try:
    schema_context = context_mgr.get_schema_context(role)
    
    if schema_context:
        print("\n[SUCESSO] Contexto gerado:")
        print(schema_context[:500] + "... (truncado)")
    else:
        print("[AVISO] Contexto vazio.")
except Exception as e:
    print(f"[ERRO] Falha no Context Manager dentro do Debug Full: {e}")

## 3. Teste do Agente 'Logistics' Isolado (LLM + Tool)
Aqui testamos se o LLM consegue receber esse contexto e formular uma query.

In [0]:
from app.agents import Agent
from app.tools import SparkSQLTool

tool = SparkSQLTool(spark)
logistics_agent = Agent("LogisticsDebug", "logistics", context_mgr, tool)

print("Executando Agente de Logística...")
# Prompt simples de verificação
try:
    response = logistics_agent.run("Conte quantas linhas existem na tabela orders (olist_dataset.olist_sales.orders).")
    print("\n=== Resposta Final ===")
    print(response)
except Exception as e:
    print(f"[ERRO] Falha na execução do Agente: {e}")

## 4. Teste do Agente 'Finance' Isolado

In [0]:
finance_agent = Agent("FinanceDebug", "finance", context_mgr, tool)

print("Executando Agente de Finanças...")
response_fin = finance_agent.run("Qual o total de pagamentos registrados na tabela de pagamentos?")

print("\n=== Resposta Final ===")
print(response_fin)